In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Portfolio Analysis").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
from pyspark.sql.functions import *

#stock_lst = ["AAPL","ABBV","ABT","ADBE","ADP","AMGN","AMT","AMZN","AROW","ARTNA","AXP","AZN","BA", \
#             "BABA","BAC","BCAL","BHP","BKNG","BTI","CAT","CECO","CHEF","CL","CMRX","COCO","CRM", \
#             "CRML","CSCO","CTOS", "CVX","DEO","DIS","ELLO","EQIX","ETN","FAT","GCMG","GE","GOOGL", \
#             "GRC","GS","GWRS","HIMX","HIPO","HON","IBM","ICHR","ISRG","JNJ","JPM","KLG","KMB","KO", \
#             "LIN","LLY","LMT","MA","MCD","MDLZ","META","MS","MSFT","MXL","NFLX","NKE","NSPR","NTES", \
#             "NVDA","ORCL","PCYO","PDD","PFE","PG","PLCE","PM","PNNT","PNST","RIO","SAP","SIRI","SNY", \
#             "SPOT","SRG","SUP","TCI","TH","TJX","TM","TSLA", "TSM","UAN","UL","UNH","UNP","V","VZ","WFC", \
#             "WMG","WMT","YTRA"]

stock_lst = ['AAPL', 'META', 'DIS', 'TSLA', 'VZ', 'ISRG', 'MSFT','NFLX','IBM','WMT']

stockdata =spark.sql("select * from stocks")

stockdata = stockdata.withColumn("date",to_date(stockdata.date,"yyyy-MM-dd"))



In [4]:
# stocks with closing price datewise
stockscp_df=stockdata.groupBy("date").pivot("symbol", stock_lst).sum("adjClose")
#stockscp_df=stockscp_df.dropna()
stockscp_df.show()

+----------+------+------+------+------+-----+------+------+----------+------+-----+
|      date|  AAPL|  META|   DIS|  TSLA|   VZ|  ISRG|  MSFT|      NFLX|   IBM|  WMT|
+----------+------+------+------+------+-----+------+------+----------+------+-----+
|2021-10-11|140.46|324.78|172.16|263.98|43.88|324.18|287.36|627.039978|118.53|44.62|
|2021-01-27|139.07|271.58|161.75|288.05|44.83|252.91|225.98|523.280029| 98.33|45.46|
|2023-06-22| 185.8| 284.3|  87.8|264.61|32.83|327.62|337.07|422.480011|124.73|51.19|
|2020-08-24|122.99|270.83|129.67|134.28|47.42|231.91| 206.8|488.809998| 99.47|41.35|
|2022-03-28|173.18|223.13|137.63|363.95|43.55|294.95|304.63| 378.51001|117.47|47.05|
|2021-08-27|146.15|371.87|178.73|237.31|45.52|345.51|292.72|558.919983|116.02|46.85|
|2021-06-22|131.58|338.33|172.14| 207.9|46.33|300.65|258.81|508.820007|120.42|43.66|
|2023-05-22|173.08|247.81|  91.1|188.87|32.94|313.62|318.69| 363.01001|121.24|48.84|
|2020-07-24| 90.34|230.24|116.69| 94.47|45.26|225.11|194.34|480.4

In [5]:
filepath='file:////home/talentum/shared/Project/Project_codes/source/SP500.csv'

stockdata1=spark.read.option("header",True).csv(filepath)
stockdata1 = stockdata1.withColumn("date",to_date(stockdata1.Date,"MM/dd/yyyy"))
stockdata1=stockdata1.dropna()

In [6]:
#SP500 with closing price datewise
sp500_df=stockdata1.select("date","Close")
sp500_df=sp500_df.withColumnRenamed("Close","SP500")
sp500_df=sp500_df.dropna()
sp500_df.show()

+----------+-------+
|      date|  SP500|
+----------+-------+
|2024-08-12|5344.39|
|2024-08-09|5344.16|
|2024-08-08|5319.31|
|2024-08-07|5199.50|
|2024-08-06|5240.03|
|2024-08-05|5186.33|
|2024-08-02|5346.56|
|2024-08-01|5446.68|
|2024-07-31|5522.30|
|2024-07-30|5436.44|
|2024-07-29|5463.54|
|2024-07-26|5459.10|
|2024-07-25|5399.22|
|2024-07-24|5427.13|
|2024-07-23|5555.74|
|2024-07-22|5564.41|
|2024-07-19|5505.00|
|2024-07-18|5544.59|
|2024-07-17|5588.27|
|2024-07-16|5667.20|
+----------+-------+
only showing top 20 rows



In [7]:
#Creating returns Dataframe

from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import functions as F

returns_df_sp500=stockscp_df.join(sp500_df,"date")
#returns_df_sp500.show()
my_window = Window.partitionBy().orderBy("date")
for col_name in returns_df_sp500.columns:
    if col_name!='date':
        returns_df_sp500 = returns_df_sp500.withColumn(col_name, F.when(F.isnull(col(col_name) - F.lag(col(col_name)).over(my_window)), 0) \
                                                       .otherwise(col(col_name) - F.lag(col(col_name)).over(my_window))/F.lag(col(col_name)).over(my_window))
for c in returns_df_sp500.columns:
    if c!='date':
        returns_df_sp500 = returns_df_sp500.withColumn(c, F.round(c, 6))

returns_df_sp500=returns_df_sp500.dropna()
returns_df_sp500=returns_df_sp500.drop('date')
returns_df_sp500.show()

returns_df=returns_df_sp500.drop("SP500")
returns_df.show()

+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|     AAPL|     META|      DIS|     TSLA|       VZ|     ISRG|     MSFT|     NFLX|      IBM|      WMT|    SP500|
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|-0.029674|-0.046368|-0.030341|-0.065731|-0.011625| -0.03388| -0.03011|-0.042174|-0.033499|-0.011432|-0.029293|
|-0.005097| 0.016059| 0.004198| -0.01776| 0.010351|-0.001199|-0.002262|  -0.0112| 0.005154| 0.061169| 0.002464|
| 0.023566| 0.006037| 0.013376| 0.019471| 0.006286|-0.008822| 0.018372| 0.023803| 0.013974| 0.002581| 0.014426|
| 0.018819| 0.013474|  6.75E-4| 0.031378| 0.006247|  1.21E-4| 0.016736| 0.021731| 0.009518| 0.007437| 0.012106|
|      0.0|-0.012703|-0.001199|-0.003968|-0.012877| 0.004662|-0.008306|-0.033583|-0.015125|-0.015616|-0.007915|
| 0.010808|-0.001417| 0.004727|-0.022576|  0.00559|  0.00693| 0.011116|-0.003947| 0.009473| -2.88E-4| 0.

In [8]:
from pyspark.mllib.stat import Statistics
import pandas as pd
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

In [9]:
def stocks_correlation(stocks_df,cols):
    # convert to vector column first
    vector_col = "corr_features"
    assembler = VectorAssembler(inputCols=cols, outputCol=vector_col)
    stocks_df_vector = assembler.transform(stocks_df).select(vector_col)

    # get correlation matrix
    matrix = Correlation.corr(stocks_df_vector, vector_col).collect()[0][0]
    corrmatrix = matrix.toArray().tolist()

    corrmatrix_df = spark.createDataFrame(corrmatrix,cols)

    for c in corrmatrix_df.columns:
        corrmatrix_df = corrmatrix_df.withColumn(c,F.round(c,2))

    stock_list = corrmatrix_df.columns #["AAPL","KO","NFLX","DIS","IBM","VZ","WMT","GE","TSLA","MA","AMZN","MSFT","UN","V"]

    # Create a DataFrame from the list of values
    values_df = spark.createDataFrame([Row(new_column=v) for v in stock_list])

    corrmatrix_df = corrmatrix_df.withColumn("row_index", monotonically_increasing_id())
    values_df = values_df.withColumn("row_index", monotonically_increasing_id())

    # Join the DataFrames on the row index
    corr_df = corrmatrix_df.join(values_df, on="row_index").drop("row_index")

    # Show the resulting DataFrame
    corr_df=corr_df.withColumnRenamed("new_column","stockName")
    return corr_df

In [10]:
corr_final_df = stocks_correlation(returns_df,returns_df.columns)
corr_final_df.show()

+----+----+----+----+----+----+----+----+----+----+---------+
|AAPL|META| DIS|TSLA|  VZ|ISRG|MSFT|NFLX| IBM| WMT|stockName|
+----+----+----+----+----+----+----+----+----+----+---------+
|0.45|0.39| 1.0|0.32|0.31|0.46|0.48|0.36|0.42|0.24|      DIS|
| 1.0|0.56|0.45|0.49|0.29|0.58|0.75|0.46|0.41|0.37|     AAPL|
|0.49|0.33|0.32| 1.0|0.05|0.37|0.45|0.36|0.17|0.14|     TSLA|
|0.37|0.22|0.24|0.14|0.33|0.28|0.38|0.23| 0.3| 1.0|      WMT|
|0.56| 1.0|0.39|0.33|0.19|0.48|0.62| 0.5|0.26|0.22|     META|
|0.58|0.48|0.46|0.37|0.29| 1.0|0.61|0.38|0.44|0.28|     ISRG|
|0.75|0.62|0.48|0.45|0.27|0.61| 1.0|0.49|0.42|0.38|     MSFT|
|0.46| 0.5|0.36|0.36|0.13|0.38|0.49| 1.0|0.13|0.23|     NFLX|
|0.29|0.19|0.31|0.05| 1.0|0.29|0.27|0.13|0.43|0.33|       VZ|
|0.41|0.26|0.42|0.17|0.43|0.44|0.42|0.13| 1.0| 0.3|      IBM|
+----+----+----+----+----+----+----+----+----+----+---------+



In [11]:
def highmoderateCorrelation(stock_df,stock_list):
    st_lst = stock_list
    portfolios = []
    for st in st_lst:
        protfolio_st = stock_df[[st,"stockName"]]
        stock_df.createOrReplaceTempView("aapstock")
        query = "select stockName from aapstock where {} between 0.5 and 0.99".format(st) # moderate -0.5 to 0.69 & high 0.7 to 0.99
        portfolio_stocks = spark.sql(query)
        portfolio_i = portfolio_stocks.select("stockName").rdd.flatMap(lambda x:x).collect()
        portfolio_i.append(st)
        portfolios.append(portfolio_i)
    return portfolios

In [12]:
correlated_portfolios = highmoderateCorrelation(corr_final_df,stock_lst)

In [13]:
correlated_portfolios

[['META', 'ISRG', 'MSFT', 'AAPL'],
 ['AAPL', 'MSFT', 'NFLX', 'META'],
 ['DIS'],
 ['TSLA'],
 ['VZ'],
 ['AAPL', 'MSFT', 'ISRG'],
 ['AAPL', 'META', 'ISRG', 'MSFT'],
 ['META', 'NFLX'],
 ['IBM'],
 ['WMT']]

In [14]:
portfolios = []
for port in correlated_portfolios:
    if 2 <= len(port) <= 3:
        portfolios.append(port)

In [15]:
portfolios

[['AAPL', 'MSFT', 'ISRG'], ['META', 'NFLX']]

In [16]:
#portfolios=[ ['AAPL', 'DIS', 'TSLA', 'VZ'],['MSFT', 'ISRG', 'META']]

In [17]:
#portfolios

In [18]:
port_df = spark.createDataFrame([(row,) for row in portfolios], ["values"])
port_df_exploded = port_df.select(explode(col("values")).alias("value"))
unique_port_df = port_df_exploded.distinct()
unique_port =unique_port_df.rdd.flatMap(lambda x :x).collect()

In [19]:
unique_port

['AAPL', 'META', 'NFLX', 'ISRG', 'MSFT']

In [20]:
# Names to Portfolios
names = []

for i in range(1, len(portfolios)+1):
    names.append('Portfolio'+str(i))


In [21]:
names

['Portfolio1', 'Portfolio2']

In [22]:
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.sql.functions import sqrt
from pyspark.mllib.linalg import DenseMatrix

"""
Returns the Annualised Expected Return of a portfolio.
Annualises the return using the 'crude' method.
"""
def getPortReturns():
    portfolio_returns = []
    for port,weight in zip(portfolios,portfolio_weightage):
           
        num_stocks=len(port)
        # creating weight matrix  
        weight_matrix= DenseMatrix(1,num_stocks, weight)
        # Creating Row matrix for the Dense matrix to do dot_product using multiply function
        weight_row_mat = RowMatrix(sc.parallelize(weight_matrix.toArray().tolist()))
        # Extracting mean return of the portfolio
        returns_df_mean_list1=[]
        returns_df_mean_list=[]
        for p in port:
            returns_df_mean_list = returns_df_mean.filter(returns_df_mean.Stocks == p).select("Mean").rdd.flatMap(lambda x: x).collect()
            returns_df_mean_list1.append(returns_df_mean_list[0])
                
        returns_df_matrix=DenseMatrix(len(returns_df_mean_list1),1,returns_df_mean_list1)
        # calculating dot_product of weight and mean return to get the expected return
        exp_ret_portfolio=weight_row_mat.multiply(returns_df_matrix).rows.collect()
        # annual expected return through crude method
        exp_ret_portfolio_annual = exp_ret_portfolio[0][0] * 250
        # Returning the lsit of expected return of portfolios
        portfolio_returns.append(exp_ret_portfolio_annual)
    
    return portfolio_returns
    
    
    
    

In [23]:
# Calculate the risk of each portfolio
from pyspark.mllib.linalg import DenseMatrix
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.sql.functions import sqrt
import numpy as np
# arguments are weight and return_df of portfolios #df_temp = return_df[port]
def getPortRisk(weights,df_temp):
    portfolio_risks = []
    
    # calculating the variance covariance matrix of return_df of portfolio
    
    returns_rdd = df_temp.rdd.map(list)
    returns_row_mat = RowMatrix(returns_rdd)
    vcv_matrix=returns_row_mat.computeCovariance()
   
    # Variance of the portfolio = weight matrix dot_product variance-covariance matricx dot_product weight transpose matrix
    weight_matrix= DenseMatrix(1, num_stocks, weights) # weight matrix
    weight_matrix_t= DenseMatrix(num_stocks, 1, weights) # weight matrix Transpose
    weight_row_mat = RowMatrix(sc.parallelize(weight_matrix.toArray().tolist()))
    vcv_w=weight_row_mat.multiply(vcv_matrix) # dot_product of weight matrix and vcv matrix
    var_portfolio = vcv_w.multiply(weight_matrix_t).rows.collect() # dot_product of (weight matrix and vcv matrix in prev line) with weight transpose
    
    # calculating standard deviation from variance and compute annual sd of the portfolio
    var_portfolio=var_portfolio[0][0]
    sd_portfolio=np.sqrt(var_portfolio)
    sd_portfolio_annual = sd_portfolio * np.sqrt(250)
    
    # returning the portfolio risk i.e. standard deviation computed annually
    portfolio_risks.append(sd_portfolio_annual)
    
    return portfolio_risks

In [24]:
returns_df_unique=returns_df[unique_port]

In [25]:
returns_df_unique.show()

+---------+---------+---------+---------+---------+
|     AAPL|     META|     NFLX|     ISRG|     MSFT|
+---------+---------+---------+---------+---------+
|-0.029674|-0.046368|-0.042174| -0.03388| -0.03011|
|-0.005097| 0.016059|  -0.0112|-0.001199|-0.002262|
| 0.023566| 0.006037| 0.023803|-0.008822| 0.018372|
| 0.018819| 0.013474| 0.021731|  1.21E-4| 0.016736|
|      0.0|-0.012703|-0.033583| 0.004662|-0.008306|
| 0.010808|-0.001417|-0.003947|  0.00693| 0.011116|
| -7.78E-4|-0.008189|-0.002955|-0.001975|-0.007229|
|-0.046304|-0.023559|-0.018489|-0.031362|-0.031857|
| 0.018972| 0.014657| 0.012147| 0.013434| 0.015434|
|-0.011211| 0.005223|-0.013391| 0.008002|  0.00216|
| 0.006682| 0.002542| 0.002543| 0.012545|-0.001386|
| 0.016895| 0.020949| 0.017171| 0.013826| 0.018888|
|-0.001187|   5.4E-4| -0.01021| 0.006258|-0.001892|
|-0.014653|-0.017648|-0.015183|-0.034028|-0.013191|
| 0.017082| 0.026041| 0.007708| 0.004616| 0.011677|
| 0.019364| 0.020079| 0.005934| 0.024061| 0.017617|
|      0.0|-

In [26]:
# Calculating the weightage of the stocks for the portfolio by minimising risks
from scipy.optimize import minimize
import numpy as np

portfolio_weightage=[]
for port in portfolios:

    num_stocks = len(returns_df_unique[port].columns)  # being the number of stocks (this is a 'global' variable)
    init_weights = [1 / num_stocks] * num_stocks  # initialise weights (x0) [Intially assume equal weightge]
    args=returns_df_unique[port] # second argument of getPortRisk() function
    bounds = tuple((0, 1) for i in range(num_stocks)) # weightage will be between 0 and 1
    cons = ({'type' : 'eq', 'fun' : lambda x : np.sum(x) - 1}) # sum of weightage will be 1
    # getting the optimised weightage through minimise() function with getPortRisk() as objective function and applying bounadries and constraints as adefined above
    results = minimize(fun=getPortRisk, x0=init_weights,args=args, bounds=bounds, constraints=cons)
    # converting the values to float and rounding off to 2 decimal places
    rounded_result = np.round(results.x, decimals=2)
    print(rounded_result)
    temp_rdd=sc.parallelize(rounded_result)
    temp_rdd=temp_rdd.map(lambda x:float(x))
    
    portfolio_weightage.append(temp_rdd.collect())


[0.31 0.46 0.23]
[0.52 0.48]


In [27]:
portfolio_weightage

[[0.31, 0.46, 0.23], [0.52, 0.48]]

In [28]:
# Calculating the investment amount of the portfolio i.e. latest closing price

latest_date = stockscp_df.select(max("date").alias("latest_date")).collect()[0]["latest_date"]
portfolio_amount = []

for port in portfolios:
    amount_rdd=stockscp_df.filter(stockscp_df.date == latest_date).select(port).rdd.flatMap(lambda x: x).map(lambda x: float(x))
    total_amount= amount_rdd.collect()
    portfolio_amount.append(total_amount)
    
    

In [29]:
print(latest_date)

2024-08-12


In [30]:
portfolio_amount

[[217.53, 406.81, 465.53], [515.95, 633.14]]

In [31]:
# calculating mean return for each stocks and creating a dtaframe
data1=unique_port
data2=[]
for c in unique_port:
    data2.append(returns_df.select(mean(col(c))).collect()[0][0]) 
data = list(zip(data1, data2))
returns_df_mean=spark.createDataFrame(data,["Stocks","Mean"])
returns_df_mean = returns_df_mean.withColumn("Mean", round(returns_df_mean["Mean"], 6))
returns_df_mean.show()

+------+--------+
|Stocks|    Mean|
+------+--------+
|  AAPL|0.001362|
|  META|0.001207|
|  NFLX| 9.91E-4|
|  ISRG| 0.00104|
|  MSFT|0.001078|
+------+--------+



In [32]:
# function call for returns calulation as per thw portflio weightage
portfolio_return_wt=getPortReturns()


In [33]:
portfolio_return_wt

[0.289325, 0.27582999999999996]

In [34]:
# Calculating beta_value with Linear Regression 

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

assembler = VectorAssembler(inputCols=["SP500"], outputCol="features")
data_with_features = assembler.transform(returns_df_sp500)
betaValues=[]
for col in returns_df_unique.columns:
# Create a LinearRegression model and fit it to the data
    lr = LinearRegression(featuresCol="features", labelCol=col) # passing each stock to LR as labelCol
    model = lr.fit(data_with_features)
    betaValues.append(model.coefficients[0]) # model.coefficients[0] gives beta value of specific stock 

print(betaValues)


[1.1899104241635308, 1.3104979650057893, 1.0270714721688912, 1.2164159809104333, 1.1761692932090735]


In [35]:
#Creating dataframe for Stocks with beta value
d1=unique_port
d2=[float(x) for x in betaValues]
d=zip(d1,d2)
beta_df=spark.createDataFrame(d,["Stocks","Beta"])
beta_df=beta_df.withColumn("Beta",round("Beta",2))
beta_df.show()

+------+----+
|Stocks|Beta|
+------+----+
|  AAPL|1.19|
|  META|1.31|
|  NFLX|1.03|
|  ISRG|1.22|
|  MSFT|1.18|
+------+----+



In [36]:
# Grouping the beta values for the portfolio

beta_port=[]
for port in portfolios:
    beta_list=[]
    for p in port:
        beta_rdd=beta_df.filter(beta_df.Stocks == p).select("Beta").rdd.flatMap(lambda x: x).map(lambda x: float(x))
        beta_list.append(beta_rdd.collect())
        beta_list_rdd=sc.parallelize(beta_list)
        beta_list_f = beta_list_rdd.flatMap(lambda x: x).collect()
    beta_port.append(beta_list_f)
        
print(beta_port)

[[1.19, 1.18, 1.22], [1.31, 1.03]]


In [37]:
# Calculating the final Portfolio Dataframe incorporating Names, Portfolios, Weightage,return,Amount,Beta values

portfolio_return_wt=[float(x) for x in portfolio_return_wt]

data=list(zip(names,portfolios,portfolio_weightage,portfolio_return_wt,portfolio_amount,beta_port))
schema = StructType([
    StructField("Names", StringType(), True),
    StructField("Portfolios", ArrayType(StringType()), True),
    StructField("Weightage", ArrayType(FloatType()), True),
    StructField("TotalReturns", FloatType(), True),
    StructField("Amount", ArrayType(FloatType()), True),
    StructField("BetaPort", ArrayType(FloatType()), True)])
                
final_portfolio_df = spark.createDataFrame(data, schema)

final_portfolio_df.show()

+----------+------------------+------------------+------------+--------------------+------------------+
|     Names|        Portfolios|         Weightage|TotalReturns|              Amount|          BetaPort|
+----------+------------------+------------------+------------+--------------------+------------------+
|Portfolio1|[AAPL, MSFT, ISRG]|[0.31, 0.46, 0.23]|    0.289325|[217.53, 406.81, ...|[1.19, 1.18, 1.22]|
|Portfolio2|      [META, NFLX]|      [0.52, 0.48]|     0.27583|    [515.95, 633.14]|      [1.31, 1.03]|
+----------+------------------+------------------+------------+--------------------+------------------+



In [38]:
# Calculating the Investment amount and beta value as per the  weightage

final_portfolio_df = final_portfolio_df.withColumn("InvestmentAmount", expr("aggregate(zip_with(Weightage, Amount, (x, y) -> x * y), 0D, (acc, x) -> acc + x)"))

final_portfolio_df=final_portfolio_df.drop("Amount")

final_portfolio_df = final_portfolio_df.withColumn("PortfolioBeta", expr("aggregate(zip_with(Weightage, BetaPort, (x, y) -> x * y), 0D, (acc, x) -> acc + x)"))

final_portfolio_df=final_portfolio_df.drop("BetaPort")

final_portfolio_df = final_portfolio_df.withColumn("TotalReturns", round(final_portfolio_df["TotalReturns"], 2)) \
.withColumn("InvestmentAmount", round(final_portfolio_df["InvestmentAmount"], 2)) \
.withColumn("PortfolioBeta", round(final_portfolio_df["PortfolioBeta"], 2))

final_portfolio_df.show(truncate=False)

+----------+------------------+------------------+------------+----------------+-------------+
|Names     |Portfolios        |Weightage         |TotalReturns|InvestmentAmount|PortfolioBeta|
+----------+------------------+------------------+------------+----------------+-------------+
|Portfolio1|[AAPL, MSFT, ISRG]|[0.31, 0.46, 0.23]|0.29        |361.64          |1.19         |
|Portfolio2|[META, NFLX]      |[0.52, 0.48]      |0.28        |572.2           |1.18         |
+----------+------------------+------------------+------------+----------------+-------------+



In [39]:
# Incorporate Volatility based on beta value , Beta VAlue > 1 High Volatile else Less Volatile

from pyspark.sql import functions as F

final_portfolio_df = final_portfolio_df.withColumn("Volatility", F.when(final_portfolio_df["PortfolioBeta"] > 1, "High") \
                                                   .otherwise("Low"))
final_portfolio_df=final_portfolio_df.drop("PortfolioBeta")
final_portfolio_df.show()




+----------+------------------+------------------+------------+----------------+----------+
|     Names|        Portfolios|         Weightage|TotalReturns|InvestmentAmount|Volatility|
+----------+------------------+------------------+------------+----------------+----------+
|Portfolio1|[AAPL, MSFT, ISRG]|[0.31, 0.46, 0.23]|        0.29|          361.64|      High|
|Portfolio2|      [META, NFLX]|      [0.52, 0.48]|        0.28|           572.2|      High|
+----------+------------------+------------------+------------+----------------+----------+



In [40]:
df_exploded1 = final_portfolio_df.select("Names","TotalReturns","InvestmentAmount","Volatility",posexplode("Portfolios").alias("pos", "Portfolios"))
df_exploded2 = final_portfolio_df.select("Names","TotalReturns","InvestmentAmount","Volatility", posexplode("Weightage").alias("pos", "Weightage"))

df_final = df_exploded1.join(df_exploded2, on=["Names","TotalReturns","InvestmentAmount","Volatility", "pos"]) \
.select("Names", "Portfolios", "Weightage","TotalReturns","InvestmentAmount","Volatility")

df_sorted = df_final.sort("Names")
df_sorted.show()

+----------+----------+---------+------------+----------------+----------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|
+----------+----------+---------+------------+----------------+----------+
|Portfolio1|      MSFT|     0.46|        0.29|          361.64|      High|
|Portfolio1|      ISRG|     0.23|        0.29|          361.64|      High|
|Portfolio1|      AAPL|     0.31|        0.29|          361.64|      High|
|Portfolio2|      META|     0.52|        0.28|           572.2|      High|
|Portfolio2|      NFLX|     0.48|        0.28|           572.2|      High|
+----------+----------+---------+------------+----------------+----------+



In [41]:
df_sorted_fin=df_sorted.filter(df_sorted.Weightage != 0)

In [42]:
df_sorted_fin.show()

+----------+----------+---------+------------+----------------+----------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|
+----------+----------+---------+------------+----------------+----------+
|Portfolio1|      ISRG|     0.23|        0.29|          361.64|      High|
|Portfolio1|      MSFT|     0.46|        0.29|          361.64|      High|
|Portfolio1|      AAPL|     0.31|        0.29|          361.64|      High|
|Portfolio2|      NFLX|     0.48|        0.28|           572.2|      High|
|Portfolio2|      META|     0.52|        0.28|           572.2|      High|
+----------+----------+---------+------------+----------------+----------+



In [43]:
filepath='file:////home/talentum/shared/Project/Project_codes/source/company_names.csv'

company_df=spark.read.option("header",True).csv(filepath)

In [44]:
company_df.show()

+------+--------------------+
|symbol|             company|
+------+--------------------+
|  AAPL|          Apple Inc.|
|  ABBV|         AbbVie Inc.|
|   ABT| Abbott Laboratories|
|  ADBE|          Adobe Inc.|
|   ADP|Automatic Data Pr...|
|  AMGN|          Amgen Inc.|
|   AMT|American Tower Co...|
|  AMZN|    Amazon.com, Inc.|
|  AROW|Arrow Financial C...|
| ARTNA|Artesian Resource...|
|   AXP|American Express ...|
|   AZN|     AstraZeneca PLC|
|    BA|  The Boeing Company|
|  BABA|Alibaba Group Hol...|
|   BAC|Bank of America C...|
|  BCAL|Southern Californ...|
|   BHP|   BHP Group Limited|
|  BKNG|Booking Holdings ...|
|   BTI|British American ...|
|   CAT|    Caterpillar Inc.|
+------+--------------------+
only showing top 20 rows



In [45]:
df_fin=df_sorted_fin.join(company_df,df_sorted_fin.Portfolios==company_df.symbol,"inner")
df_fin=df_fin.drop("symbol")
df_fin.show()

+----------+----------+---------+------------+----------------+----------+--------------------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|             company|
+----------+----------+---------+------------+----------------+----------+--------------------+
|Portfolio1|      MSFT|     0.46|        0.29|          361.64|      High|Microsoft Corpora...|
|Portfolio1|      AAPL|     0.31|        0.29|          361.64|      High|          Apple Inc.|
|Portfolio1|      ISRG|     0.23|        0.29|          361.64|      High|Intuitive Surgica...|
|Portfolio2|      META|     0.52|        0.28|           572.2|      High|Meta Platforms, Inc.|
|Portfolio2|      NFLX|     0.48|        0.28|           572.2|      High|       Netflix, Inc.|
+----------+----------+---------+------------+----------------+----------+--------------------+



In [46]:
df_fin.write.mode("overwrite").saveAsTable("Portfolios")

In [47]:
portfoliodata=spark.sql("select * from Portfolios")

In [48]:
portfoliodata.show()
stockdata.show()

+----------+----------+---------+------------+----------------+----------+--------------------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|             company|
+----------+----------+---------+------------+----------------+----------+--------------------+
|Portfolio1|      MSFT|     0.46|        0.29|          361.64|      High|Microsoft Corpora...|
|Portfolio1|      AAPL|     0.31|        0.29|          361.64|      High|          Apple Inc.|
|Portfolio1|      ISRG|     0.23|        0.29|          361.64|      High|Intuitive Surgica...|
|Portfolio2|      META|     0.52|        0.28|           572.2|      High|Meta Platforms, Inc.|
|Portfolio2|      NFLX|     0.48|        0.28|           572.2|      High|       Netflix, Inc.|
+----------+----------+---------+------------+----------------+----------+--------------------+

+------+--------+-------+--------------+-------------+-------+----------+--------+-------------+---------+-------+----------------+----

In [49]:
joineddf = portfoliodata.join(stockdata, portfoliodata["Portfolios"] == stockdata["symbol"], "inner")
joineddf=joineddf.drop("label")
joineddf.show()

+----------+----------+---------+------------+----------------+----------+--------------------+------+--------+-------+--------------+-------------+------+----------+------+--------+-------+----------------+--------+--------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|             company|symbol|adjClose| change|changeOverTime|changePercent| close|      date|  high|     low|   open|unadjustedVolume|  volume|    vwap|
+----------+----------+---------+------------+----------------+----------+--------------------+------+--------+-------+--------------+-------------+------+----------+------+--------+-------+----------------+--------+--------+
|Portfolio1|      MSFT|     0.46|        0.29|          361.64|      High|Microsoft Corpora...|  MSFT|  406.81|  -0.25|  -6.141601E-4|  -0.06141601|406.81|2024-08-12|408.76|404.2434| 407.06|        14541173|13790473|  406.42|
|Portfolio1|      MSFT|     0.46|        0.29|          361.64|      High|Microsoft Corpora...| 

In [50]:
joineddf.write.mode("overwrite").saveAsTable("PortfoliosDetail")

In [51]:
joineddf.show()

+----------+----------+---------+------------+----------------+----------+--------------------+------+--------+-------+--------------+-------------+------+----------+------+--------+-------+----------------+--------+--------+
|     Names|Portfolios|Weightage|TotalReturns|InvestmentAmount|Volatility|             company|symbol|adjClose| change|changeOverTime|changePercent| close|      date|  high|     low|   open|unadjustedVolume|  volume|    vwap|
+----------+----------+---------+------------+----------------+----------+--------------------+------+--------+-------+--------------+-------------+------+----------+------+--------+-------+----------------+--------+--------+
|Portfolio1|      MSFT|     0.46|        0.29|          361.64|      High|Microsoft Corpora...|  MSFT|  406.81|  -0.25|  -6.141601E-4|  -0.06141601|406.81|2024-08-12|408.76|404.2434| 407.06|        14541173|13790473|  406.42|
|Portfolio1|      MSFT|     0.46|        0.29|          361.64|      High|Microsoft Corpora...| 

In [52]:
filepath='file:////home/talentum/csvfile_demo'

#dffile=spark.sql("select * from PortfoliosDetail")
# Write DataFrame to CSV file with header
#sd.write.mode("overwrite").format("csv").save(filepath)

joineddf.coalesce(1).write.mode("overwrite").option("header","true").format("csv").save(filepath)